At this point, you should have completed step 1, and have yolo-training-data

In [20]:
# Config
SPLITS = {"train": 0.7, "valid": 0.2, "test": 0.1}
INPUT_DIRECTORY = "yolo-training-data"

In [21]:
# imports
import random
import os

In [22]:
def split(input_directory: str, splits: dict[str, float]):
    images_dir = os.path.join(input_directory, "images")
    labels_dir = os.path.join(input_directory, "labels")
    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]
    random.shuffle(image_files)

    total = len(image_files)
    split_points = {}
    last = 0
    for k, v in splits.items():
        count = int(v * total)
        split_points[k] = (last, last + count)
        last += count
    # Adjust last split to include any remainder
    last_key = list(splits.keys())[-1]
    split_points[last_key] = (split_points[last_key][0], total)

    for split_name, (start, end) in split_points.items():
        split_img_dir = os.path.join(input_directory, split_name, "images")
        split_lbl_dir = os.path.join(input_directory, split_name, "labels")
        os.makedirs(split_img_dir, exist_ok=True)
        os.makedirs(split_lbl_dir, exist_ok=True)
        for img_file in image_files[start:end]:
            # Copy image
            src_img = os.path.join(images_dir, img_file)
            dst_img = os.path.join(split_img_dir, img_file)
            os.replace(src_img, dst_img)
            # Copy label (assume same filename but .txt extension)
            label_file = os.path.splitext(img_file)[0] + ".txt"
            src_lbl = os.path.join(labels_dir, label_file)
            if os.path.exists(src_lbl):
                dst_lbl = os.path.join(split_lbl_dir, label_file)
                os.replace(src_lbl, dst_lbl)

In [23]:
# GO!
split(INPUT_DIRECTORY, SPLITS)
os.rmdir(os.path.join(INPUT_DIRECTORY, "images"))
os.rmdir(os.path.join(INPUT_DIRECTORY, "labels"))